In [1]:
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import numpy as np

In [2]:
# Player Count

purchase = pd.read_csv('purchase_data.csv')
purchase.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [15]:
## Total Number of Players

total_players = len(purchase.SN.unique())
total_players_df = {'Total Players': [total_players]}
total_players_df = pd.DataFrame(total_players_df, columns=['Total Players'])
total_players_df

,Total Players
0,576


In [60]:
# Purchasing Analysis (Total)

## Number of Unique Items
unique_items = len(purchase['Item Name'].unique())
unique_items

## Average Purchase Price
avg_price = purchase.Price.mean()
round(avg_price,2)

## Total Number of Purchases
total_purchases = len(purchase['Purchase ID'].unique())
total_purchases

## Total Revenue
total_revenue = purchase.Price.sum()
total_revenue

purchase_analysis = {'Unique Items': [unique_items],
                     'Average Purchase Price': [round(avg_price,2)],
                     'Total Number of Purchases': [total_purchases],
                     'Total Revenue': [total_revenue]}
purchase_analysis = pd.DataFrame(purchase_analysis, columns=['Unique Items', 
                                                             'Average Purchase Price', 
                                                             'Total Number of Purchases', 
                                                             'Total Revenue'])
purchase_analysis


,Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [61]:
# Gender Demographics

### Filter by unique IDs
genders = ['Female', 'Male', 'Other / Non-Disclosed']
gender_grouped = purchase.groupby(['Gender'])['SN'].unique()

## Percentage and Count of Female Players
## Percentage and Count of Male Players
## Percentage and Count of Other / Non-Disclosed Players
for i in genders:
    gender_grouped[i] = len(gender_grouped[i])

gender_grouped = pd.DataFrame(gender_grouped)
gender_grouped = gender_grouped.rename(columns={'SN': 'Total Count'})
gender_grouped['Percentage'] = gender_grouped['Total Count']/total_players *100
pd.options.display.float_format = '{:,.2f}%'.format

gender_grouped

,Total Count,Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [62]:
# Purchasing Analysis (Gender)

# The below each broken by gender

## Purchase Count
## Average Purchase Price
## Total Purchase Value
genders_df = purchase[['Gender', 'Price']]
pd.options.display.float_format = '${:,.2f}'.format

summary_gender = genders_df.groupby(['Gender']).agg([np.count_nonzero, np.mean, np.sum]) 

summary_gender.columns = summary_gender.columns.droplevel()
summary_gender.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
summary_gender['Avg Total Purchase per Person'] = summary_gender['Total Purchase Value']/gender_grouped['Total Count']
summary_gender['Purchase Count'] = summary_gender['Purchase Count'].astype(int)

summary_gender


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [52]:
# Age Demographics
pd.options.display.float_format = '{:,.2f}%'.format

# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
bins = [0, 10, 15, 20, 25, 30, 35, 40, 45]

group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44']

purchase["Age Group"] = pd.cut(purchase["Age"], bins, labels=group_names)
age_grouped = purchase.groupby(['Age Group'])['SN'].unique()
for i in group_names:
    age_grouped[i] = len(age_grouped[i])
age_grouped = pd.DataFrame(age_grouped)
age_grouped['Percentage'] = age_grouped['SN']/total_players *100
age_grouped = age_grouped.rename(columns={'SN': 'Total Count'})
age_grouped

,Total Count,Percentage
Age Group,,
<10,24,4.17%
10-14,41,7.12%
15-19,150,26.04%
20-24,232,40.28%
25-29,59,10.24%
30-34,37,6.42%
35-39,26,4.51%
40-44,7,1.22%


In [63]:
pd.options.display.float_format = '${:,.2f}'.format

## Purchase Count
## Average Purchase Price
## Total Purchase Value
age_df = purchase[["Age Group", 'Price']]

summary_age = age_df.groupby("Age Group").agg([np.count_nonzero, np.mean, np.sum])
summary_age.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
summary_age['Avg Total Purchase per Person'] = summary_age['Total Purchase Value']/age_grouped['Total Count']
summary_age['Purchase Count'] = summary_age['Purchase Count'].astype(int)

summary_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,32,$3.40,$108.96,$4.54
10-14,54,$2.90,$156.60,$3.82
15-19,200,$3.11,$621.56,$4.14
20-24,325,$3.02,$981.64,$4.23
25-29,77,$2.88,$221.42,$3.75
30-34,52,$2.99,$155.71,$4.21
35-39,33,$3.40,$112.35,$4.32
40-44,7,$3.08,$21.53,$3.08


In [54]:
# Top Spenders

# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
top_5_spenders = purchase.groupby('SN')['Price'].sum().nlargest(5)
top_5_spenders = top_5_spenders.index.values.tolist()
spenders_df = purchase[purchase.SN.isin(top_5_spenders)]
spenders_df = spenders_df[['SN', 'Price']]

spenders_df.head()


,SN,Price
74,Lisosia93,$4.64
120,Lisosia93,$3.81
128,Iral74,$4.14
148,Iskadarya95,$4.03
222,Chamjask73,$4.23


In [64]:
## SN
## Purchase Count
## Average Purchase Price
## Total Purchase Value
summary_spenders = spenders_df.groupby('SN').agg([np.count_nonzero, np.mean, np.sum])
summary_spenders.columns = summary_spenders.columns.droplevel()
summary_spenders.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
summary_spenders['Purchase Count'] = summary_spenders['Purchase Count'].astype(int)

summary_spenders


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Chamjask73,3,$4.61,$13.83
Idastidru52,4,$3.86,$15.45
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
Lisosia93,5,$3.79,$18.96


In [56]:
# Most Popular Items

# Identify the 5 most popular items by purchase count, then list (in a table):
top_5_items = purchase.groupby('Item ID')['Item Name'].count().nlargest(5)
top_5_items = top_5_items.index.values.tolist()
items_df = purchase[purchase['Item ID'].isin(top_5_items)]
items_df = items_df[['Item ID', 'Item Name', 'Price']]

items_df.head()


,Item ID,Item Name,Price
0,108,"Extraction, Quickblade Of Trembling Hands",$3.53
18,82,Nirvana,$4.90
25,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23
56,108,"Extraction, Quickblade Of Trembling Hands",$3.53
61,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23


In [65]:
## Item ID
## Item Name
## Purchase Count
## Item Price
## Total Purchase Value

summary_items = items_df.groupby(['Item ID', 'Item Name']).agg([np.count_nonzero, np.mean, np.sum])
summary_items.columns = summary_items.columns.droplevel()
summary_items = summary_items.reset_index()
summary_items.columns = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']
summary_items['Purchase Count'] = summary_items['Purchase Count'].astype(int)

summary_items


,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
1,82,Nirvana,9,$4.90,$44.10
2,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
3,145,Fiery Glass Crusader,9,$4.58,$41.22
4,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76


In [58]:
# Most Profitable Items

# Identify the 5 most profitable items by total purchase value, then list (in a table):
top_5_profits = purchase.groupby('Item ID')['Price'].sum().nlargest(5)
top_5_profits = top_5_profits.index.values.tolist()
profits_df = purchase[purchase['Item ID'].isin(top_5_profits)]
profits_df = profits_df[['Item ID', 'Item Name', 'Price']]

profits_df.head()


,Item ID,Item Name,Price
2,92,Final Critic,$4.88
18,82,Nirvana,$4.90
25,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23
61,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23
62,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23


In [66]:
## Item ID
## Item Name
## Purchase Count
## Item Price
## Total Purchase Value

summary_profits = profits_df.groupby(['Item ID', 'Item Name']).agg([np.count_nonzero, np.mean, np.sum])
summary_profits = summary_profits.reset_index()
summary_profits.columns = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']
summary_profits['Purchase Count'] = summary_profits['Purchase Count'].astype(int)

summary_profits


,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,82,Nirvana,9,$4.90,$44.10
1,92,Final Critic,8,$4.88,$39.04
2,103,Singed Scalpel,8,$4.35,$34.80
3,145,Fiery Glass Crusader,9,$4.58,$41.22
4,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
